# Importation

In [1]:
import geopandas as gpd
from osgeo import gdal
import os
import numpy as np
from my_function import (
    pre_traitement_img,
    supprimer_dossier_non_vide)
import sys
sys.path.append('/home/onyxia/work/libsigma')
import read_and_write as rw

In [2]:
racine = "/home/onyxia/work"

# Création image toute bande

In [5]:
## Partie Serie_temp_S2_allbands.tif
print("Partie pretraitement image")
#  Définition des paramètres 

print("Définitions des paramètres")
input_raster_dir = os.path.join(racine,"data/images")
l_images = sorted(os.listdir(input_raster_dir))
if ".keep" in l_images:
    l_images.remove(".keep")
shapefile_path = os.path.join(racine,"data/project/emprise_etude.shp")

output_dir = os.path.join(racine,"output_pretraitement")
os.makedirs(output_dir, exist_ok=True)

masque_path = os.path.join(racine,"Depot_Git/results/data/img_pretraitees/masque_foret.tif")

Partie pretraitement image
Définitions des paramètres


In [6]:
# Réalisation des pré-traitements sur les images individuelles 
pre_traitement_img(
    p_emprise = shapefile_path,
    l_images = l_images,
    input_raster_dir = input_raster_dir,
    output_dir = output_dir)

Chargement du geojson en str
Chargement du geojson en str ok!
Traitements des images


/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Image 1/60 traitée
Image 2/60 traitée
Image 3/60 traitée
Image 4/60 traitée
Image 5/60 traitée
Image 6/60 traitée
Image 7/60 traitée
Image 8/60 traitée
Image 9/60 traitée
Image 10/60 traitée
Image 11/60 traitée
Image 12/60 traitée
Image 13/60 traitée
Image 14/60 traitée
Image 15/60 traitée
Image 16/60 traitée
Image 17/60 traitée
Image 18/60 traitée
Image 19/60 traitée
Image 20/60 traitée
Image 21/60 traitée
Image 22/60 traitée
Image 23/60 traitée
Image 24/60 traitée
Image 25/60 traitée
Image 26/60 traitée
Image 27/60 traitée
Image 28/60 traitée
Image 29/60 traitée
Image 30/60 traitée
Image 31/60 traitée
Image 32/60 traitée
Image 33/60 traitée
Image 34/60 traitée
Image 35/60 traitée
Image 36/60 traitée
Image 37/60 traitée
Image 38/60 traitée
Image 39/60 traitée
Image 40/60 traitée
Image 41/60 traitée
Image 42/60 traitée
Image 43/60 traitée
Image 44/60 traitée
Image 45/60 traitée
Image 46/60 traitée
Image 47/60 traitée
Image 48/60 traitée
Image 49/60 traitée
Image 50/60 traitée
Image 51/

In [ ]:
# Charger le vecteur avec Geopandas
emprise = gpd.read_file(shapefile_path).to_crs("EPSG:2154")

# Extraire le GeoJSON sous forme de string
print("Chargement du geojson en str")
geojson_str = emprise.to_json()
print("Chargement du geojson en str ok!")

print("Traitements des images")
for i,img in enumerate(L_images) :
    date = img[11:19]
    bande = img[53:]
    ds_img = rw.open_image(os.path.join(input_raster_dir,img))
    name_file = f"traitement_{date}_{bande}"
    output_file = os.path.join(output_dir,name_file)
    # Appliquer le clip avec GDAL
    resolution = 10  # Résolution (10 m)
    output_raster_test = gdal.Warp(
        output_file, # Chemin de fichier, car on utilise GTiff
        # "",  # Pas de chemin de fichier, car on utilise MEM
        ds_img,  # Fichier en entrée (chemin ou objet gdal)
        format = "GTiff", # Utiliser GTiff comme format
        # format = "MEM",  # Utiliser MEM comme format
        cutlineDSName = geojson_str,  # Passer directement le GeoJSON
        cropToCutline = True,
        outputType = gdal.GDT_UInt16, #UInt16
        dstSRS = "EPSG:2154",  # Reprojection
        xRes = resolution,  # Résolution X
        yRes = resolution,  # Résolution Y
        dstNodata = 0  # Valeur NoData
    )
    print(f"Image {i+1}/{len(L_images)} traitée")
ds_img = None
emprise = None
geojson_str = None

In [ ]:
# Construction array avec masque pour un nombre de bandes voulues
ref_raster_path = os.path.join(output_dir,"traitement_20220125_B2.tif")
L_images_clip = sorted(os.listdir(output_dir))
x,y = rw.get_image_dimension(rw.open_image(ref_raster_path))[:2]
bandes = 10
array_tot = np.zeros((x,y,bandes))

masque = rw.load_img_as_array(masque_path)

# Initialisation de la liste pour stocker les arrays masqués
L_array_masqued = []

# Parcourir toutes les images de L_images_clip
for i, img in enumerate(L_images_clip[:10]):
    path = os.path.join(output_dir, img)
    array = rw.load_img_as_array(path)
    array_masqued = np.where(masque == 1, array, 0)
    L_array_masqued.append(array_masqued)

# Concaténation des arrays masqués
print("Concaténation en cours")
array_final_masqued = np.concatenate(L_array_masqued, axis=2)
print("Tableau concaténé avec masque appliqué")

# Save array into image
out_masqued = f"/home/onyxia/work/Depot_Git/results/data/img_pretraitees/Serie_temp_S2_{bandes}_bands.tif"
print("Ecriture en cours")
rw.write_image(out_filename=out_masqued, array=array_final_masqued, data_set=rw.open_image(ref_raster_path))
print("Ecriture terminée")

In [ ]:
# Construction array avec masque
ref_raster_path = os.path.join(output_dir,"traitement_20220125_B2.tif")
L_images_clip = sorted(os.listdir(output_dir))
x,y = rw.get_image_dimension(rw.open_image(ref_raster_path))[:2]
bandes = 60
array_tot = np.zeros((x,y,bandes))

masque = rw.load_img_as_array(masque_path)

# Initialisation de la liste pour stocker les arrays masqués
L_array_masqued = []

# Parcourir toutes les images de L_images_clip
for i, img in enumerate(L_images_clip):
    path = os.path.join(output_dir, img)
    array = rw.load_img_as_array(path)
    array_masqued = np.where(masque == 1, array, 0)
    L_array_masqued.append(array_masqued)

# Concaténation des arrays masqués
print("Concaténation en cours")
array_final_masqued = np.concatenate(L_array_masqued, axis=2)
print("Tableau concaténé avec masque appliqué")

# Save array into image
out_masqued = "/home/onyxia/work/Depot_Git/results/data/img_pretraitees/Serie_temp_S2_allbands.tif"
print("Ecriture en cours")
rw.write_image(out_filename=out_masqued, array=array_final_masqued, data_set=rw.open_image(ref_raster_path))
print("Ecriture terminée")

# Serie temporelle de NDVI

In [8]:
# Definitions des paramètres
traitements_dir = "/home/onyxia/work/output_pretraitement"
L_traitements = [os.path.join(traitements_dir,i) for i in sorted(os.listdir(traitements_dir))]
# L_traitements = os.path.join(traitements_dir,sorted(os.listdir(traitements_dir)))
ref_raster_path = os.path.join(traitements_dir,"traitement_20220125_B2.tif")
masque_path = "/home/onyxia/work/Depot_Git/results/data/img_pretraitees/masque_foret.tif"
masque = rw.load_img_as_array(masque_path)

```
Attention le ndvi est compris entre -1 et 1 et doit être sauvegardé en float32 donc ça peut être négatif
Tu peux utiliser ça aussi pour te débrouiller avec la division par 0 :
    # Calculate NDVI, handling potential division by zero
    ndvi = np.divide(nir - red, nir + red, out=np.zeros_like(nir - red), where=(nir + red) != 0)
    # To avoid division by 0, a common approach is to output 0 where both NIR and Red are 0.
```

In [ ]:
# Cellule de test pour date 20220125

red_n = os.path.join(output_dir,L_traitements[4])
nir_n = os.path.join(output_dir,L_traitements[8])

red,nir = rw.load_img_as_array(red_n).astype(int),rw.load_img_as_array(nir_n).astype(int)

nominator = nir-red
nominator_masked = np.where(nominator>=0,nominator,0)

denominator = nir+red

ndvi = np.where(denominator!=0,nominator_masked/denominator,0)

ndvi_masked = np.where(masque == 1, ndvi, 0)

rw.write_image(
    out_filename="/home/onyxia/work/ndvi_test.tif",
    array=ndvi_masked, 
    data_set=rw.open_image(ref_raster_path),
    gdal_dtype=gdal.GDT_Float32
    )

In [9]:
# Pour les 6 dates 
x,y = rw.get_image_dimension(rw.open_image(ref_raster_path))[:2]
bandes = 6

dates = ["20220125","20220326","20220405","20220714","20220922","20221111"] # Liste des 6 dates
nir_name = 'B8.'
r_name = 'B4.'

ndvi_blank = np.zeros((x,y,bandes), dtype=np.float32)  # Créer un array NDVI avec les mêmes dimensions que nir

In [ ]:
np.unique(masque)

In [ ]:
for i,date in enumerate(dates) :
    print(i,date)
    for img in L_traitements:
        if date in img and r_name in img :
            red = rw.load_img_as_array(img)[:,:,0].astype('float32')
            print (f"Bande rouge date {date}")
        if date in img and nir_name in img :
            nir = rw.load_img_as_array(img)[:,:,0].astype('float32')
            print (f"Bande infra-rouge date {date}")
    nominator = nir-red
    nominator_masked = np.where(nominator>=0,nominator,0)
    denominator = nir+red
    ndvi_blank[:,:,i] = np.where(denominator!=0,nominator_masked/denominator,0)
ndvi_masked = np.where(masque == 1, ndvi_blank, -9999)

In [ ]:
# Save array into image
out_ndvi = "/home/onyxia/work/Depot_Git/results/data/img_pretraitees/Serie_temp_S2_ndvi.tif"
print("Ecriture en cours")
rw.write_image(out_filename=out_ndvi, array=ndvi_masked, data_set=rw.open_image(ref_raster_path), gdal_dtype=gdal.GDT_Float32)
print("Ecriture terminée")

# Suppression dossier output

In [2]:
#Supp du dossier output pour faire de la place

supprimer_dossier_non_vide(output_dir)

NameError: name 'output_dir' is not defined